In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Remove existing installations
!pip uninstall -y torch torchvision opacus psutil pycryptodome cryptography

# Install necessary packages
!pip install --quiet torch torchvision opacus psutil pycryptodome cryptography

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: psutil 5.9.5
Uninstalling psutil-5.9.5:
  Successfully uninstalled psutil-5.9.5
Found existing installation: cryptography 43.0.3
Uninstalling cryptography-43.0.3:
  Successfully uninstalled cryptography-43.0.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
# Restart the runtime to ensure new installations are loaded correctly
import os
os.kill(os.getpid(), 9)

In [ ]:
# After restarting, run this cell:
import torch
import opacus
from opacus import PrivacyEngine

In [ ]:
# @title Default title text
# Verify versions and CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"Opacus version: {opacus.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.5.1+cu124
Opacus version: 1.5.2
CUDA available: True


In [ ]:
# Check number of CPU cores
from psutil import cpu_count
print("Number of CPU: ", cpu_count())
!cat /proc/cpuinfo

Number of CPU:  12
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 7
microcode	: 0xffffffff
cpu MHz		: 2200.156
cache size	: 39424 KB
physical id	: 0
siblings	: 12
core id		: 0
cpu cores	: 6
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp ibrs_enhanced fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat avx512_vnni md_clear arch_capabilities
bugs		: spectre_v1 spectre_v2 spec_sto

In [ ]:
# Show GPU info
!nvidia-smi

Wed Dec  4 01:32:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              44W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# Test CUDA setup
device = torch.device("cuda")
print(f"\nUsing device: {device}")
print(f"Device properties:")
print(f"  Name: {torch.cuda.get_device_name(0)}")
print(f"  Memory allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
print(f"  Memory cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")


Using device: cuda
Device properties:
  Name: NVIDIA A100-SXM4-40GB
  Memory allocated: 0.00 MB
  Memory cached: 0.00 MB


In [ ]:
# Try a minimal DP setup
model = torch.nn.Linear(10, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
privacy_engine = PrivacyEngine()

model.to(device)
print("\nBasic DP setup successful")

/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(



Basic DP setup successful


In [ ]:
from google.colab import files

# Upload the dpsgd_rng_benchmark.py and model.py
uploaded = files.upload()

Saving dpsgd_rng_benchmark.py to dpsgd_rng_benchmark.py
Saving aes_prng.py to aes_prng.py
Saving model.py to model.py


In [ ]:
!ls *.py

aes_prng.py  dpsgd_rng_benchmark.py  model.py


In [ ]:
from dpsgd_rng_benchmark import main

main(output_dir = "/content/drive/MyDrive/", model_type = "resnet", poisson_sampling = True, privacy_engine_type = "aes")
main(output_dir = "/content/drive/MyDrive/", model_type = "resnet", poisson_sampling = True, privacy_engine_type = "standard")


Running resnet benchmark WITH poisson subsampling
Secure mode activated...
Initializing privacy engine of type <class 'aes_prng.AESPrivacyEngine'>
Creating dummy dataset...
Setting up private model and optimizer...
Private model setup complete.

Warmup...

DPSGD Settings:
Privacy Engine noise_multiplier: 1.0
Optimizer noise_multiplier: 1.0


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)



DPSGD_NO_NOISE Settings:
Privacy Engine noise_multiplier: 0.0
Optimizer noise_multiplier: 0.0

Noise Generation std parameter: 1.0
Generator: <aes_prng.HWAccelAESGenerator object at 0x7da825a2ca00>
Total time: 0.783ms

Running main benchmark...
Iteration 0/1000
Iteration 100/1000
Iteration 200/1000
Iteration 300/1000
Iteration 400/1000
Iteration 500/1000
Iteration 600/1000
Iteration 700/1000
Iteration 800/1000
Iteration 900/1000

SAMPLING MODE: POISSON


Benchmark Results:
--------------------------------------------------------------------------------

DPSGD:
Average step time: 265.05 ms (± 194.58 ms)
Throughput: 120.73 examples/sec

Memory Usage:
CPU Memory: 1185.16 MB

Peak Memory Usage:
Peak CPU Memory: 0.00 MB

DPSGD_NO_NOISE:
Average step time: 42.04 ms (± 4.24 ms)
Throughput: 761.12 examples/sec

Memory Usage:
CPU Memory: 1182.43 MB

Peak Memory Usage:
Peak CPU Memory: 0.00 MB

Forward/Backward Pass and Optimizer Timing Statistics:
----------------------------------------------

/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


Initializing privacy engine of type <class 'opacus.privacy_engine.PrivacyEngine'>
Creating dummy dataset...
Setting up private model and optimizer...
Private model setup complete.

Warmup...

DPSGD Settings:
Privacy Engine noise_multiplier: 1.0
Optimizer noise_multiplier: 1.0

Noise Generation std parameter: 1.0
Generator: None
Total time: 0.133ms

DPSGD_NO_NOISE Settings:
Privacy Engine noise_multiplier: 0.0
Optimizer noise_multiplier: 0.0

Running main benchmark...
Iteration 0/1000
Iteration 100/1000
Iteration 200/1000
Iteration 300/1000
Iteration 400/1000
Iteration 500/1000
Iteration 600/1000
Iteration 700/1000
Iteration 800/1000
Iteration 900/1000

SAMPLING MODE: POISSON


Benchmark Results:
--------------------------------------------------------------------------------

DPSGD:
Average step time: 41.91 ms (± 4.91 ms)
Throughput: 763.57 examples/sec

Memory Usage:
CPU Memory: 1249.95 MB

Peak Memory Usage:
Peak CPU Memory: 0.00 MB

DPSGD_NO_NOISE:
Average step time: 41.04 ms (± 4.3

In [ ]:
main(output_dir = "/content/drive/MyDrive/", model_type = "transformer", poisson_sampling = True, privacy_engine_type = "aes")
main(output_dir = "/content/drive/MyDrive/", model_type = "transformer", poisson_sampling = True, privacy_engine_type = "standard")


Running transformer benchmark WITH poisson subsampling
Secure mode activated...
Initializing privacy engine of type <class 'aes_prng.AESPrivacyEngine'>
Creating dummy dataset...
Setting up private model and optimizer...
Private model setup complete.

Warmup...

DPSGD Settings:
Privacy Engine noise_multiplier: 1.0
Optimizer noise_multiplier: 1.0

DPSGD_NO_NOISE Settings:
Privacy Engine noise_multiplier: 0.0
Optimizer noise_multiplier: 0.0

Noise Generation std parameter: 1.0
Generator: <aes_prng.HWAccelAESGenerator object at 0x7da8247bf380>
Total time: 2.644ms

Running main benchmark...
Iteration 0/1000
Iteration 100/1000
Iteration 200/1000
Iteration 300/1000
Iteration 400/1000
Iteration 500/1000
Iteration 600/1000
Iteration 700/1000
Iteration 800/1000
Iteration 900/1000

SAMPLING MODE: POISSON


Benchmark Results:
--------------------------------------------------------------------------------

DPSGD:
Average step time: 57.75 ms (± 33.83 ms)
Throughput: 554.10 examples/sec

Memory Usa